In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# 1. Load the data

In [3]:
data=pd.read_csv('C:/Users/tsb7592/Downloads/Basic_Desc_output.csv')

In [4]:
data['PIN']=data['PIN'].str.upper()

In [5]:
data.head()

,PIN,DeviceID,Assessment Name,Inst,ItemID,Response,Score,Position,ResponseTime,DateCreated,InstStarted,InstEnded,Age_month,Age_group
0,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 08:30:42,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
1,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 08:30:43,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
2,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL13,2,0.00,3,NaN,2021-12-05 08:32:53,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
3,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL10,1,1.00,4,NaN,2021-12-05 08:32:59,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
4,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL15,2,0.00,5,NaN,2021-12-05 08:33:18,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"


# 2. Filter the inst to be IBQ

In [6]:
df= data[data.Inst=='Infant Behavior Questionnaire - Revised - Very Short Form']

In [7]:
df.PIN=df.PIN.str[:5]

In [8]:
df.Response =pd.to_numeric(df.Response)
df.Score =pd.to_numeric(df.Score)

In [9]:
df.describe()

,Response,Score,Position,ResponseTime,Age_month
count,1482.00,1357.00,1482.00,1406.00,897.00
mean,4.64,4.76,20.00,13.97,373.48
std,2.10,1.89,11.26,51.86,67.90
min,1.00,1.00,1.00,0.51,257.00
25%,3.00,3.00,10.00,4.47,320.00
50%,5.00,5.00,20.00,6.43,371.00
75%,6.00,6.00,30.00,10.91,445.00
max,8.00,7.00,39.00,1071.29,493.00


# 3. Rescore all items to ensure appropriate response coding

In [10]:
import numpy as np
#The score should equal the response for all items EXCEPT SelfReg_ibqrvsh11, for which the score should be 8-response 
df['Score'] = np.where(df.ItemID=='SelfReg_ibqrvsh11',8-df.Response, df.Response)

# 4. Calculate the three composite scores

In [11]:
a= ['SelfReg_ibqrvsh1','SelfReg_ibqrvsh2',
    'SelfReg_ibqrvsh7','SelfReg_ibqrvsh8', 
    'SelfReg_ibqrvsh13','SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15',
    'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21',
    'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27' ,
    'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

b= ['SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
    'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
    'SelfReg_ibqrvsh16', 'SelfReg_ibqrvsh17',  
    'SelfReg_ibqrvsh22', 'SelfReg_ibqrvsh23',  
    'SelfReg_ibqrvsh28', 'SelfReg_ibqrvsh29', 
    'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33']


c= [ 'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6',
    'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 
    'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
    'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
    'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
      'SelfReg_ibqrvsh34', 'SelfReg_ibqrvsh35']

In [12]:
SelfReg_SurgIBQR= df[df.ItemID.isin(a)].groupby('PIN')['Score'].mean()
SelfReg_NegAffIBQR= df[df.ItemID.isin(b)].groupby('PIN')['Score'].mean()
SelfReg_EffContIBQR= df[df.ItemID.isin(c)].groupby('PIN')['Score'].mean()


In [13]:
comp =[SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR]

In [16]:
cdf = pd.concat(comp,axis=1)
cdf.columns = ['SelfReg_SurgIBQR','SelfReg_NegAffIBQR','SelfReg_EffContIBQR']

In [17]:
cdf.corr()

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00


# 5. Calculate item-rest correlations for every item – e.g., what is the Pearson correlation for SelfReg_ibqrvsh1 with the average score for the 12 other items on SelfReg_SurgIQBR. There should be 37 item-rest correlations. 

In [18]:
items=['SelfReg_ibqrvsh1', 'SelfReg_ibqrvsh2', 'SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
       'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6', 'SelfReg_ibqrvsh7',
       'SelfReg_ibqrvsh8', 'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
       'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 'SelfReg_ibqrvsh13',
       'SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15', 'SelfReg_ibqrvsh16',
       'SelfReg_ibqrvsh17', 'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
       'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21', 'SelfReg_ibqrvsh22',
       'SelfReg_ibqrvsh23', 'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
       'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27', 'SelfReg_ibqrvsh28',
       'SelfReg_ibqrvsh29', 'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
       'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33', 'SelfReg_ibqrvsh34',
       'SelfReg_ibqrvsh35', 'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

In [19]:
from scipy.stats import pearsonr

def item_rest(itemlist, li=None):
    if not li:
        li ={}
        
    for i in range(len(itemlist)):
        rest = itemlist.copy()
        rest.pop(i)
        list1 = df[df.ItemID==itemlist[i]].groupby('PIN')['Score'].mean()
        list2  = df[df.ItemID.isin(rest)].groupby('PIN')['Score'].mean()
    
    # Apply the pearsonr()
        corr, _ = pearsonr(list1, list2)
        print(itemlist[i], ': Pearsons correlation is %.3f ' % corr)
        li[itemlist[i]] = corr
    return li

In [20]:
itemrest = item_rest(a)

SelfReg_ibqrvsh1 : Pearsons correlation is 0.237 
SelfReg_ibqrvsh2 : Pearsons correlation is 0.180 
SelfReg_ibqrvsh7 : Pearsons correlation is 0.446 
SelfReg_ibqrvsh8 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh13 : Pearsons correlation is 0.196 
SelfReg_ibqrvsh14 : Pearsons correlation is 0.285 
SelfReg_ibqrvsh15 : Pearsons correlation is 0.155 
SelfReg_ibqrvsh20 : Pearsons correlation is 0.227 
SelfReg_ibqrvsh21 : Pearsons correlation is 0.242 
SelfReg_ibqrvsh26 : Pearsons correlation is 0.376 
SelfReg_ibqrvsh27 : Pearsons correlation is 0.360 
SelfReg_ibqrvsh36 : Pearsons correlation is 0.348 
SelfReg_ibqrvsh37 : Pearsons correlation is 0.197 


In [21]:
itemrest =item_rest(b, itemrest)

SelfReg_ibqrvsh3 : Pearsons correlation is 0.474 
SelfReg_ibqrvsh4 : Pearsons correlation is 0.603 
SelfReg_ibqrvsh9 : Pearsons correlation is 0.343 
SelfReg_ibqrvsh10 : Pearsons correlation is 0.265 
SelfReg_ibqrvsh16 : Pearsons correlation is 0.467 
SelfReg_ibqrvsh17 : Pearsons correlation is 0.326 
SelfReg_ibqrvsh22 : Pearsons correlation is -0.052 
SelfReg_ibqrvsh23 : Pearsons correlation is 0.548 
SelfReg_ibqrvsh28 : Pearsons correlation is 0.373 
SelfReg_ibqrvsh29 : Pearsons correlation is 0.409 
SelfReg_ibqrvsh32 : Pearsons correlation is 0.406 
SelfReg_ibqrvsh33 : Pearsons correlation is 0.448 


In [22]:
itemrest = item_rest(c,itemrest)

SelfReg_ibqrvsh5 : Pearsons correlation is 0.262 
SelfReg_ibqrvsh6 : Pearsons correlation is 0.312 
SelfReg_ibqrvsh11 : Pearsons correlation is 0.179 
SelfReg_ibqrvsh12 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh18 : Pearsons correlation is 0.530 
SelfReg_ibqrvsh19 : Pearsons correlation is 0.445 
SelfReg_ibqrvsh24 : Pearsons correlation is 0.411 
SelfReg_ibqrvsh25 : Pearsons correlation is 0.532 
SelfReg_ibqrvsh30 : Pearsons correlation is 0.437 
SelfReg_ibqrvsh31 : Pearsons correlation is 0.466 
SelfReg_ibqrvsh34 : Pearsons correlation is 0.510 
SelfReg_ibqrvsh35 : Pearsons correlation is 0.456 


In [23]:
irc = pd.DataFrame.from_dict(itemrest, orient='index', columns=['item-rest-correlation'])
irc.round(3).to_csv('C:/Users/tsb7592/Downloads/IBQ_item_rest_correlation.csv')

# 6. Describe each of the 3 composite scores overall and by age group. NOTE: the Infant Behavior Questionnaire is not applicable to all age groups. 

In [24]:
cdf=cdf.reset_index()

In [25]:
spin = cdf.PIN.to_list()
cpin = []
for pin in spin:
    pin+='C'
    cpin.append(pin)

In [26]:
len(spin)

37

In [27]:
child =data[data.PIN.isin(cpin)]

In [28]:
child.PIN=child.PIN.str[:5]

In [29]:
child[child.Age_group.notnull()]['PIN'].nunique()

33

In [30]:
de = pd.merge(cdf, child[['PIN','Age_month','Age_group']], on=['PIN'], how ='left')

In [31]:
de =de.drop_duplicates()

In [32]:
de.shape

(37, 6)

In [33]:
de.groupby(['Age_group'])['SelfReg_SurgIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",6.00,5.87,1.02,4.15,5.46,6.08,6.58,6.92
"(12, 15]",1.00,5.54,NaN,5.54,5.54,5.54,5.54,5.54
"(24, 27]",1.00,5.46,NaN,5.46,5.46,5.46,5.46,5.46
"(3, 4]",4.00,4.48,0.53,3.92,4.15,4.42,4.75,5.15
"(4, 5]",7.00,4.70,0.67,3.69,4.31,4.77,5.08,5.69
"(5, 6]",3.00,5.03,1.39,3.62,4.35,5.08,5.73,6.38
"(6, 8]",7.00,4.45,0.54,3.92,4.08,4.46,4.54,5.54
"(8, 10]",4.00,5.23,0.74,4.23,4.98,5.35,5.60,6.00


In [34]:
de.groupby(['Age_group'])['SelfReg_NegAffIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",6.00,4.36,0.96,3.00,3.75,4.62,4.75,5.67
"(12, 15]",1.00,5.50,NaN,5.50,5.50,5.50,5.50,5.50
"(24, 27]",1.00,4.42,NaN,4.42,4.42,4.42,4.42,4.42
"(3, 4]",4.00,4.40,0.85,3.17,4.17,4.67,4.90,5.08
"(4, 5]",7.00,4.32,0.73,3.17,3.96,4.33,4.79,5.25
"(5, 6]",3.00,4.17,1.89,2.83,3.08,3.33,4.83,6.33
"(6, 8]",7.00,4.58,0.63,3.33,4.50,4.75,4.79,5.42
"(8, 10]",4.00,4.08,1.75,2.42,2.67,4.04,5.46,5.83


In [35]:
de.groupby(['Age_group'])['SelfReg_EffContIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",6.00,5.50,1.27,3.83,4.75,5.29,6.52,7.08
"(12, 15]",1.00,5.67,NaN,5.67,5.67,5.67,5.67,5.67
"(24, 27]",1.00,5.83,NaN,5.83,5.83,5.83,5.83,5.83
"(3, 4]",4.00,6.12,0.34,5.83,5.83,6.12,6.42,6.42
"(4, 5]",7.00,5.18,0.72,4.25,4.83,5.08,5.38,6.50
"(5, 6]",3.00,5.31,0.38,4.92,5.12,5.33,5.50,5.67
"(6, 8]",7.00,5.06,0.61,4.25,4.62,5.25,5.29,6.08
"(8, 10]",4.00,5.35,1.03,4.08,4.90,5.38,5.83,6.58


# 7. Add the other Self-Regulation instruments to the data file – that is, go back to the item-level data and select the Inst = “Playing with Bubbles” and “Toy Out of Reach”  

In [36]:
df2= data[(data.Inst=='Playing with Bubbles')]
df3 = data[(data.Inst=='Toy Out of Reach')]
df3.shape

(1804, 14)

# 8. Correlate the 3 composites with PA_2 and PA_3 from “Playing with Bubbles”  

In [37]:
pa2 =df2[df2.ItemID=='PA_2'][['PIN','Score']]
pa3 =df2[df2.ItemID=='PA_3'][['PIN','Score']]

In [38]:
pa2

,PIN,Score
105,TX009C,0.00
406,TX005C,2.00
951,APPLETONC1,3.00
1321,MO008C,1.00
1596,MN011C,3.00
...,...,...
49007,WI017C,2.00
49795,MO022C,1.00
50726,TX027C,3.00
50727,TX027C,3.00


In [39]:
PA = pd.merge(pa2,pa3, on='PIN')
PA.columns =['PIN','PA_2','PA_3']

In [40]:
cdf.corrwith(PA['PA_2'])

SelfReg_SurgIBQR      -0.12
SelfReg_NegAffIBQR    -0.01
SelfReg_EffContIBQR   -0.25
dtype: float64

In [41]:
c2 =cdf.corr().append(cdf.corrwith(PA['PA_2']), ignore_index=True)

In [42]:
c23 =c2.append(cdf.corrwith(PA['PA_3']), ignore_index=True)

In [44]:
c23

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00
PA_2,-0.12,-0.01,-0.25
PA_3,-0.17,-0.02,-0.29


# 9. Mean-level scores for 'toy out of reach'

In [45]:
df3.ItemID.unique()

array(['LabT_Title', 'SelfReg_LabTAB_Toy_Intro',
       'SelfReg_LabTAB_Toy_Baseline', 'SelfReg_Toy_Trial1Intro',
       'SelfReg_Toy_frus_reg1', 'SelfReg_Toy_frus_escalation1',
       'SelfReg_Toy_frus_cope1', 'SelfReg_Toy_Trial2Intro',
       'SelfReg_Toy_frus_reg2', 'SelfReg_Toy_frus_escalation2',
       'SelfReg _Toy_frus_cope2', 'SelfReg_Toy_Trial3Intro',
       'SelfReg_Toy_frus_reg3', 'SelfReg _Toy_frus_escalation3',
       'SelfReg_Toy _frus_cope3'], dtype=object)

In [46]:
#9a
f1 = df3[df3['ItemID'].str.contains("frus_reg")].groupby('PIN')['Score'].mean().reset_index()

In [47]:
#9b
f2= df3[df3['ItemID'].str.contains("frus_escal")].groupby('PIN')['Score'].mean().reset_index()

In [48]:
#9c
f3=df3[df3['ItemID'].str.contains("frus_cope")].groupby('PIN')['Score'].mean().reset_index()
f3

,PIN,Score
0,APPLETONC1,3.00
1,M018C,1.00
2,MN002C,2.00
3,MN004C,0.67
4,MN006C,1.00
...,...,...
89,WI030C,3.00
90,WI031C,2.00
91,WI032C,1.00
92,WI033C,1.33


In [49]:
f12 =pd.merge(f1,f2, on='PIN')
frus =pd.merge(f12,f3, on='PIN')

In [66]:
frus.columns =['PIN', 'frus_reg', 'frus_escalation', 'frus_cope']

In [67]:
new =pd.merge(PA, frus,on='PIN', how='left')

In [71]:
cf =c23.append(cdf.corrwith(new['frus_reg']), ignore_index=True)
cf1 =cf.append(cdf.corrwith(new['frus_escalation']), ignore_index=True)
cf2 =cf1.append(cdf.corrwith(new['frus_cope']), ignore_index=True)
cf2

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
0,1.00,0.40,0.41
1,0.40,1.00,0.04
2,0.41,0.04,1.00
3,-0.12,-0.01,-0.25
4,-0.17,-0.02,-0.29
5,-0.24,-0.25,0.06
6,0.11,0.08,-0.01
7,-0.02,-0.01,0.09


In [72]:
cf2.index=['SelfReg_SurgIBQR', 'SelfReg_NegAffIBQR', 'SelfReg_EffContIBQR','PA_2','PA_3','frus_reg', 'frus_escalation', 'frus_cope']

In [73]:
cf2

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00
PA_2,-0.12,-0.01,-0.25
PA_3,-0.17,-0.02,-0.29
frus_reg,-0.24,-0.25,0.06
frus_escalation,0.11,0.08,-0.01
frus_cope,-0.02,-0.01,0.09


In [74]:
cf2.round(2).to_csv('C:/Users/tsb7592/Downloads/IBQ_corr.csv')

In [75]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/tsb7592/Downloads/IBQ_composite_scores.xlsx', engine='xlsxwriter')

de.groupby(['Age_group'])['SelfReg_SurgIBQR'].describe().round(2).to_excel(writer, sheet_name='SurgIBQR')

de.groupby(['Age_group'])['SelfReg_NegAffIBQR'].describe().round(2).to_excel(writer, sheet_name='NegAffIBQR')

de.groupby(['Age_group'])['SelfReg_EffContIBQR'].describe().round(2).to_excel(writer, sheet_name='EffContIBQR')

# Close the Pandas Excel writer and output the Excel file.
writer.save()